In [1]:
import flask
from flask import Flask
import VibeCheckBot
import os

import atexit
from apscheduler.schedulers.background import BackgroundScheduler

import tweepy
#import pandas as pd
import json
import sys
import requests
import logging
import re, string, random
import nltk
nltk.download('omw-1.4')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier

nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
stop_words = stopwords.words('english')

import tweepy
import json
import requests
import logging
import time



In [5]:
#%pip install pipreqs
import pipreqs

pipreqs /Users/alexs/DataWithAlex_Github/Twitter/VibeCheckBot

NameError: name 'Users' is not defined

In [1]:
# BIRTH THE VIBE CHECK MODEL

import tweepy
#import pandas as pd
import json
import sys
import tweepy
import json
import requests
import logging
import re, string, random
import nltk
nltk.download('omw-1.4')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier

nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
stop_words = stopwords.words('english')

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

if __name__ == "__main__":

    positive_tweets = twitter_samples.strings('positive_tweets.json')
    negative_tweets = twitter_samples.strings('negative_tweets.json')
    text = twitter_samples.strings('tweets.20150430-223406.json')
    tweet_tokens = twitter_samples.tokenized('positive_tweets.json')[0]

    stop_words = stopwords.words('english')

    positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
    negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

    positive_cleaned_tokens_list = []
    negative_cleaned_tokens_list = []

    for tokens in positive_tweet_tokens:
        positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    for tokens in negative_tweet_tokens:
        negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    all_pos_words = get_all_words(positive_cleaned_tokens_list)

    freq_dist_pos = FreqDist(all_pos_words)
    print(freq_dist_pos.most_common(10))

    positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
    negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

    positive_dataset = [(tweet_dict, "Positive")
                         for tweet_dict in positive_tokens_for_model]

    negative_dataset = [(tweet_dict, "Negative")
                         for tweet_dict in negative_tokens_for_model]

    dataset = positive_dataset + negative_dataset

    random.shuffle(dataset)

    train_data = dataset[:7000]
    test_data = dataset[7000:]

    classifier = NaiveBayesClassifier.train(train_data)

    print("Accuracy is:", classify.accuracy(classifier, test_data))

    print(classifier.show_most_informative_features(10))

    custom_tweet = "I ordered just once from TerribleCo, they screwed up, never used the app again."

    custom_tokens = remove_noise(word_tokenize(custom_tweet))

    print(custom_tweet, classifier.classify(dict([token, True] for token in custom_tokens)))

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\alexs\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alexs\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\alexs\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexs\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]
Accuracy is: 0.996
Most Informative Features
                      :) = True           Positi : Negati =    984.4 : 1.0
                     sad = True           Negati : Positi =     26.7 : 1.0
                    glad = True           Positi : Negati =     24.3 : 1.0
                follower = True           Positi : Negati =     19.4 : 1.0
                     x15 = True           Negati : Positi =     17.0 : 1.0
               community = True           Positi : Negati =     15.0 : 1.0
                     bro = True           Positi : Negati =     13.0 : 1.0
                  arrive = True           Positi : Negati =     11.9 : 1.0
                   didnt = True           Negati : Positi =     11.7 : 1.0
                 welcome = True           Positi : Negati =     11.2 : 1.0
None
I ordered just once from TerribleCo, they screwed 

In [6]:
%pip install flask

     ---------------------------------------- 95.2/95.2 KB 1.1 MB/s eta 0:00:00
     -------------------------------------- 224.9/224.9 KB 2.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\alexs\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [9]:
def get_vibe(tweet_text):
    custom_tweet = tweet_text
    custom_tokens = remove_noise(word_tokenize(custom_tweet))
    return(classifier.classify(dict([token, True] for token in custom_tokens)))


In [10]:
## THIS CODE WORKS TO RESPOND TO TWEET 
def get_vibe(tweet_text):
    custom_tweet = tweet_text
    custom_tokens = remove_noise(word_tokenize(custom_tweet))
    return(classifier.classify(dict([token, True] for token in custom_tokens)))

client = tweepy.Client(consumer_key="YUwWeeXzpnQaYQYYXbISj7SRx",
                       consumer_secret="8mGpxCb2SSJSELk3OrUWO2oOh78JqnhGJ1rVIW20QFEW5m0DaQ",
                       access_token="1519770072142827525-ocVARinf1ccoz5GFMTfrUgqMGxY422",
                       access_token_secret="vrZQimplvz6SCOMDMQcVmHQ6nTY3dm4pH28RsA0JdcyWS",
                       bearer_token='AAAAAAAAAAAAAAAAAAAAAF2JcgEAAAAAapxT45Goa6gNW8ZhwE2Mi2Bztik%3DcdRisSszsBs7zCnTBP90JwPIoxcN4pRymLTcnRQE5NS5Hbh8SJ')


alex = client.get_user(username='VibeCheckBot')
alex_id = alex.data.id

#client.get_users_followers(alex_id)
mentions = client.get_users_mentions(alex_id)
tweets = mentions.data

for tweet in reversed(tweets):
        
        new_id = tweet.id
        #user_id = tweet.

        #  user = client.get_user(author_id=user_id)
        # screen_name = user.data.username

        if 'vibe check' in tweet.text:
            # logger.info("Responding back with QOD to -{}".format(mention.id))
            try:
                #tweet = get_quote()

                # media = api.media_upload("created_image.png")
                #logger.info("liking and replying to tweet")

                #reply to tweet
                x = get_vibe(tweet.text)
                client.create_tweet(in_reply_to_tweet_id = new_id, text = x)
                #client.create_tweet(quote_tweet_id = new_id, text = x)

                # api.create_favorite(mention.id)
                #api.update_status('@' + mention.user.screen_name + " Here's your Quote", mention.id,
                                 # media_ids=[media.media_id])
            except:
                print("failed")
                #logger.info("Already replied to {}".format(mention.id))
